In [ ]:
try:
    import ROOT
except:
    print('Root framework not available.')
import numpy as np
import pandas as pd
import qtconsole
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd

In [ ]:
%qtconsole

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 9]
# %matplotlib notebook

# Root data structure from FCCee Target Tracking

## Open file

In [ ]:
outFile = ROOT.TFile.Open('FCCeeTargetTracking.root', 'READ')

## Inspect structure

### File structure with ls() or GetListOfKeys()

In [ ]:
outFile.ls()

In [ ]:
for k in outFile.GetListOfKeys():
    print(k)

### Select TTree with Get()

In [ ]:
amorLeave = outFile.Get('amor_leave')
nAmorLeave = amorLeave.GetEntries()
print('Particles leaving the amorphous target: {:d}\n'.format(nAmorLeave))

### TTree strcuture with Print()

In [ ]:
amorLeave.Print()

### Default entry with Show()

In [ ]:
amorLeave.Show()

## Conversion to Python Pandas Data Frame

In [ ]:
amorLeaveRdf = ROOT.RDataFrame(amorLeave)
amorLeaveDict = amorLeaveRdf.AsNumpy()
print(amorLeaveDict.keys())
print([q.shape for q in amorLeaveDict.values()])

In [ ]:
amorLeaveMat = np.column_stack([q for q in amorLeaveDict.values()])
amorLeaveMat.shape
amorLeaveDf = pd.DataFrame(data=amorLeaveMat, columns=amorLeaveDict.keys())
amorLeaveDf['pdgId'] = amorLeaveDf['pdgId'].astype(int)
print(amorLeaveDf.columns)
print(amorLeaveDf.shape)

In [ ]:
amorLeaveDf.head()

In [ ]:
amorLeaveDf.tail()

In [ ]:
amorLeaveDf.describe()

### Select positrons

In [ ]:
positronId = 11
positronLeaveDf = amorLeaveDf[amorLeaveDf['pdgId']==positronId]

### Deprecated method for conversion

In [ ]:
amorLeaveMatDepr, amorLeaveLabelsDepr = amorLeave.AsMatrix(return_labels=True)
print(amorLeaveLabelsDepr)
print(amorLeaveMatDepr.shape)

## Import dataframe with standard format

In [ ]:
positronLeaveDf = pd.read_fwf('ex_gen1_StandardDf.txt')

## Working with the Pandas Data Frame

### Number of positrons after the target

In [ ]:
print(positronLeaveDf.columns)
print(positronLeaveDf.shape)

In [ ]:
positronLeaveDf.describe()

### Plot distribuiton

In [ ]:
pzCutoff = 25.
positronLowE = positronLeaveDf[positronLeaveDf['pz']<pzCutoff]

color = positronLeaveDf['E'].to_numpy()
color[color>pzCutoff] = pzCutoff
binEdges = np.arange(0, np.max(positronLeaveDf['pz']), 5.)

fig, ax = plt.subplots(2, 2)
ax[0,0].scatter(positronLeaveDf['x'], positronLeaveDf['y'], 1, color)
ax[1,0].hist(positronLeaveDf['x'], bins=100)
ax[1,0].hist(positronLowE['x'], bins=100)
ax[0,1].hist(positronLeaveDf['y'], bins=100, orientation='horizontal')
ax[0,1].hist(positronLowE['y'], bins=100, orientation='horizontal')
ax[1,0].set_xlim(ax[0,0].get_xlim())
ax[0,1].set_ylim(ax[0,0].get_ylim())
ax[0,0].set_xlabel('x [mm]')
ax[0,0].set_ylabel('y [mm]')
ax[0,0].yaxis.set_label_position('right')
ax[0,0].yaxis.tick_right()
ax[1,1].hist(positronLeaveDf['pz'], bins=binEdges)
ax[1,1].hist(positronLowE['pz'], bins=binEdges)
ax[1,1].set_xlabel('pz [MeV/c]')
ax[1,1].set_xlim((0, 100.))
ax[1,1].legend(('All', 'pz <= {:.1f} MeV/c'.format(pzCutoff)))
plt.show()